In [1]:
from planner.planner2_ChainOfThought import (load_scene_graph, generate_prompt, get_navigation_plan)
from data.robots2 import robots

In [2]:
scene_graph = load_scene_graph("/Users/nayunkim/Documents/GitHub/repairPlan/data/sceneGraphs/3dsg_withCOR.json")
prompt = generate_prompt(scene_graph, robots)

In [3]:
user_input = "there is a stain on the ceiling in room id 320. the robot is currently in room id 332"
navigation_plan, json_output= get_navigation_plan(user_input, scene_graph, robots)

In [4]:
print(navigation_plan)

### 1. **Identify Defect:**
   - **Reasoning:**
     - The defect described is a "stain on the ceiling" in **room 320**. Based on the 3D scene graph, the ceiling element in **room 320** is identified as **element id 569** with an `ifc_guid` of **"1T7BPQ_hn69uk5$Jp8cvMf"**.
     - The ceiling is located at a height of **3.7 meters** in **room 320**, which is critical for determining the reachability of the robots.

### 2. **Check Reachability:**
   - **Reasoning:**
     - The stain is on the ceiling, which is located at a height of **3.7 meters**.
     - We need to check which robots can reach this height. The following robots have the following maximum reach heights:
       - **Robot 1**: Max reach height = 2.0 meters (cannot reach the ceiling).
       - **Robot 2**: Max reach height = 3.0 meters (cannot reach the ceiling).
       - **Robot 3**: Max reach height = 3.7 meters (can reach the ceiling).
       - **Robot 4**: Max reach height = 2.0 meters (cannot reach the ceiling).
       

1. **Identify Defect:**
   - Reasoning: The defect is described as a stain on the ceiling located in room id 309. To identify the target defect node, I will look at the scene graph for the ceiling elements associated with room id 309. The ceiling elements are identified by their type and location. In this case, I will find the ceiling element that corresponds to room id 309.

2. **Check Reachability:**
   - The location of the stain is on the ceiling of room id 309, which has a height of 3.7 meters (as indicated by the ceiling's z-coordinate).
   - I will compare the `max_reach_height` of each robot with the height of the ceiling in room id 309 (7.85 meters).
   - The robots and their reach heights are:
     - robot1: 2500 mm (2.5 m)
     - robot2: 3000 mm (3.0 m)
     - robot3: 3500 mm (3.5 m)
     - robot4: 3700 mm (3.7 m)
     - robot5: 2800 mm (2.8 m)
     - robot6: 2300 mm (2.3 m)
     - robot7: 2200 mm (2.2 m)
     - robot8: 2600 mm (2.6 m)
     - robot9: 3200 mm (3.2 m)
     - robot10: 4000 mm (4.0 m)
   - **Reachable Robots:** 
     - robot10 (4.0 m) - can reach the ceiling
     - robot4 (3.7 m) - can just reach the ceiling
   - **Reasoning:** Robots 1, 2, 3, 5, 6, 7, 8, and 9 cannot reach the ceiling since their maximum reach heights are less than 3.7 meters. Only robots 4 and 10 can reach the stain.

3. **Generate Navigation Paths:**
   - The robot is currently in room id 314 and needs to navigate to room id 309. The path will involve navigating through doors.
   - The navigation path from room id 314 to room id 309 can be constructed by checking the doors connecting these rooms.
   - From the scene graph, I will find the doors that connect room id 314 to room id 309.
   - The path may look like this: 
     - Room 314 → Door connecting to Room 310 → Room 310 → Door connecting to Room 309 → Room 309.
   - **Path Representation:** 
     - Path: [314, door_id_1, 310, door_id_2, 309]
   - **Reasoning:** The path is constructed based on the available doors connecting the rooms. Each door must be checked for accessibility.

4. **Check Accessibility:**
   - For the navigation path, I will check the sizes of the doors against the sizes of the robots.
   - I will extract the door sizes from the scene graph and compare them with the sizes of the robots.
   - If any doors are too narrow for the robots, I will need to consider alternative paths.
   - **Accessible Robots:** 
     - Assuming the doors are wide enough for robots 4 and 10, both can access room 309.
   - **Reasoning:** If any robot cannot fit through a door, I will need to find an alternate route or check if the door sizes can accommodate the robot's width.

5. **Final Output:**
```json
{
   "defect_node": "ceiling_id_309",
   "defect_node_ifcGUID": "ceiling_ifc_guid_309",
   "selected_robots": ["robot4", "robot10"],
   "navigation_paths": [
       {
           "path": ["314", "door_id_1", "310", "door_id_2", "309"]
       }
   ],
   "reasoning": "The defect identified is a stain on the ceiling in room id 309. Robots 4 and 10 can reach the ceiling height of 3.7 meters. The navigation path from room 314 to room 309 was constructed based on the available doors connecting the rooms. The accessibility assessment indicates that both robots can fit through the doors leading to room 309."
}

In [5]:
from planner.planner_repairPlan import plan_robot_task
from data.robots import robots

import json
user_input = "there is a stain on the ceiling in room id 8. the robot is currently in room id 3"
repair_plan = plan_robot_task(json_output, user_input, robots)
print(repair_plan)

Based on the provided information, the defect is a stain on the ceiling in room 320, which is at a height of 3.7 meters. The robot needs to be able to clean the surface and reach a height of at least 3.7 meters. 

### Robot Selection:
- **Robot 3**: Max reach height is 3500 mm (3.5 meters), which is insufficient to reach the ceiling at 3.7 meters.
- **Robot 5**: Max reach height is 2800 mm (2.8 meters), which is insufficient.
- **Robot 6**: Max reach height is 2300 mm (2.3 meters), which is insufficient.
- **Robot 8**: Max reach height is 2600 mm (2.6 meters), which is insufficient.
- **Robot 9**: Max reach height is 3200 mm (3.2 meters), which is also insufficient.

None of the selected robots can reach the required height of 3.7 meters. However, **Robot 10** (which is not in the selected list but is part of the environment information) has a max reach height of 4000 mm (4 meters), which is sufficient to reach the ceiling. Additionally, Robot 10 has the "CleanSurface" action, which is